## PyTorch Tutorial 08 - Logistic Regression

### Steps in implementation

1. Design model (input size, output size, forward pass)
2. Construct loss and optimizer
3. Training loop
- Forward pass: compute y_pred
- Backward pass: gradients
- Update weights

In [2]:
import torch
import torch.nn as nn

import matplotlib as plt
import numpy as np

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### Prepare the data

In [3]:
bc = datasets.load_breast_cancer()

X, y = bc.data, bc.target
X.shape, y.shape

((569, 30), (569,))

In [4]:
n_samples, n_features = X.shape

In [5]:
# Split the data into train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [6]:
# scale the features

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
# Convert to tensors

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [10]:
# Convert row vector y to column vector

y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

### Model preparation

f = wx + b, sigmoid activation

In [11]:
class LogisticRegression(nn.Module):

    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()

        self.lin = nn.Linear(input_dim, output_dim)

    
    def forward(self,x):
        h = self.lin(x)

        y_hat = torch.sigmoid(h)
        return y_hat
    
model = LogisticRegression(n_features, 1)

In [13]:
# loss
criterion = nn.BCELoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [14]:
num_epochs = 100

for epoch in range(num_epochs):
    # forward
    y_pred = model(X_train)

    # loss
    l = criterion(y_pred, y_train)

    # backward
    l.backward()

    # update
    optimizer.step()

    # Zero grad
    optimizer.zero_grad()

    if epoch%10==0:
        [w,b] = model.parameters()
        print("Epoch {}; Loss {}".format(epoch+1, l.item()))

Epoch 1; Loss 0.7918785810470581
Epoch 11; Loss 0.6114059090614319
Epoch 21; Loss 0.5053495764732361
Epoch 31; Loss 0.4366104006767273
Epoch 41; Loss 0.3884562849998474
Epoch 51; Loss 0.3527244031429291
Epoch 61; Loss 0.32503190636634827
Epoch 71; Loss 0.302835613489151
Epoch 81; Loss 0.28456616401672363
Epoch 91; Loss 0.2692048251628876


In [21]:
with torch.no_grad(): # We need to use this so that this operation does not keep track of gradients and is detached from the main graph
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print("Accuracy {:.3f}".format(acc))

Accuracy 0.921


tensor(0.9211)
